In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

#Lahore Campus

In [ ]:
url = "http://lhr.nu.edu.pk/faculty/"
r = requests.get(url)
print(r)

<Response [200]>


In [ ]:
soup = BeautifulSoup(r.content, 'html.parser')
# print(soup.prettify())

In [ ]:
f = soup.findAll('div', class_='facultyCard')

In [ ]:
id_ = []
name = []
designation = []
HEC_approved = []
education = []
email = []
department = []
extension = []
image_URL = []

In [ ]:
pattern = r'HEC Approved PhD Supervisor'
for i in range(len(f)):
    faculty = f[i]
    try:
        id_.append(int(faculty.find('a',class_='faculty-link')['href'].split('/')[-1]))
    except:
        id_.append(-1)
    name.append(faculty.find('h5' , class_='text-center').text)
    designation.append(faculty.find('p',class_='small').text.strip().split('\n')[0])
    email.append(faculty.find('p', class_='mb-0').text)
    image_URL.append('https://lhr.nu.edu.pk'+faculty.find('img', class_='card-img-top')['src'])
    title = faculty.find('p' , class_="text-center").text
    match = re.search(pattern, title)
    if match:
        HEC_approved.append(match.group())
    else:
        HEC_approved.append("No")
    department.append(faculty.find('a', class_='faculty-link')['href'].split('/')[1])
    furl = 'https://lhr.nu.edu.pk/'+faculty.find('a', class_='faculty-link')['href']
    faculty_request = requests.get(furl)
    soup1 = BeautifulSoup(faculty_request.content, 'html.parser')
    extension.append(soup1.find('span' ,class_='small').text.split(' ')[-1])
    edu = soup1.find('div', class_='col-lg-8 col-md-6 col-sm-12 text-justify')
    try:
      education.append(edu.find('li').text.split(',')[0])
    except:
      education.append("-1")



In [ ]:
df = pd.DataFrame({'ID':id_,
                  'Name':name,
                  'Designation':designation,
                  'HEC Approved PHD':HEC_approved,
                  'Highest Education': education,
                  'Email': email,
                  'Department' :department,
                  'extension' :extension,
                  'Image URL' : image_URL})

In [ ]:
df.head()

,ID,Name,Designation,HEC Approved PHD,Highest Education,Email,Department,extension,Image URL
0,1238,Dr. Kashif Zafar,Professor & HOD,HEC Approved PhD Supervisor,Ph.D(Computer Science),kashif.zafar@nu.edu.pk,fsc,Ext:221,https://lhr.nu.edu.pk/media/Faculty/02_-_Dr._K...
1,4391,Dr. Asif Mahmood Gillani,Professor,HEC Approved PhD Supervisor,Ph.D(Computing),asif.gilani@nu.edu.pk,fsc,Ext:291,https://lhr.nu.edu.pk/media/Faculty/03_-_Dr._A...
2,5424,Dr. Hammad Naveed,Professor,HEC Approved PhD Supervisor,Ph.D. (Bioengineering),hammad.naveed@nu.edu.pk,fsc,Ext:227,https://lhr.nu.edu.pk/media/Faculty/01_-_Dr._H...
3,4027,Dr. Aamir Wali,Associate Professor,HEC Approved PhD Supervisor,PhD (CS),aamir.wali@nu.edu.pk,fsc,Ext:260,https://lhr.nu.edu.pk/media/Faculty/44_-_Dr._A...
4,6174,Dr. Arshad Ali,Associate Professor,HEC Approved PhD Supervisor,PhD (CS),arshad.ali1@nu.edu.pk,fsc,Ext:406,https://lhr.nu.edu.pk/media/Faculty/04_-_Dr._A...


In [ ]:
df.to_csv('lhr.csv')

#Peshawar Campus

In [ ]:
url_pwr = "http://pwr.nu.edu.pk/"

req = requests.get(url_pwr)
print(req)

<Response [200]>


In [ ]:
soup_pwr = BeautifulSoup(req.content, 'html.parser')
# print(soup_pwr.prettify())

In [ ]:
pwr_f = soup_pwr.find_all('ul' , class_="dropdown")

In [ ]:
depts = pwr_f[7].find_all('ul' , class_="dropdown")

In [ ]:
pwr_id_ = []
pwr_name = []
pwr_designation = []
pwr_HEC_approved = []
pwr_education = []
pwr_email = []
pwr_department = []
pwr_extension = []
pwr_image_URL = []

In [ ]:
for i in range(len(depts)):
  dept_links = "http://pwr.nu.edu.pk/"+depts[i].find('a' , string="Faculty Members")['href']
  dept_req = requests.get(dept_links)
  soup_pwr_dept = BeautifulSoup(dept_req.content, 'html.parser')
  soup_pwr_dept_f = soup_pwr_dept.find_all('div' , class_="team-members")
  for j in range(len(soup_pwr_dept_f)):
    soup_pwr_dept_faculty = soup_pwr_dept_f[j]
    pwr_image_URL.append("http://pwr.nu.edu.pk/"+soup_pwr_dept_faculty.find('img' , class_="img-fullwidth")['src']) #    img
    pwr_designation.append(soup_pwr_dept_faculty.find('h5' , class_="text-theme-color").text)#   designation
    pwr_name.append(soup_pwr_dept_faculty.find('a' , class_="text-theme-color-2").text)#   name
    pwr_dept_faculty_profile = "http://pwr.nu.edu.pk/"+soup_pwr_dept_faculty.find('a' , class_="text-theme-color-2")['href'] #profile link to send request
    pwr_id_.append(soup_pwr_dept_faculty.find('a' , class_="text-theme-color-2")['href'].split('=')[-1]) #     id
    pwr_department.append(depts[i].find('a' , string="Faculty Members")['href'].split('-')[0])#     department

    pwr_dept_faculty_profile_req = requests.get(pwr_dept_faculty_profile) #send request to server
    soup_pwr_dept_faculty_profile_f = BeautifulSoup(pwr_dept_faculty_profile_req.content, 'html.parser')

    Hec = (soup_pwr_dept_faculty_profile_f.find('h5', style=lambda style: style and 'color: green' in style).text.strip('()'))#    HEC approved try except

    if Hec=="":
      pwr_HEC_approved.append("No")
    else:
      pwr_HEC_approved.append(Hec)

    findTagEducation=soup_pwr_dept_faculty_profile_f.find('div', id='education')  #education
    res = [s.strip() for s in findTagEducation.find('li').text.split(',')]
    pwr_education.append(res[0])

    pwr_extension.append((soup_pwr_dept_faculty_profile_f.find('ul' , class_="list-inline mt-5")).find('a' , class_="text-gray").text.split('|')[-1])#     extension
    pwr_email.append((((soup_pwr_dept_faculty_profile_f.find_all('ul' , class_="list-inline mt-5")[0]).find_all('li'))[1]).find('a' , class_="text-gray").text)#   email




In [ ]:
df_pwr = pd.DataFrame({'ID':pwr_id_,
                  'Name':pwr_name,
                  'Designation':pwr_designation,
                  'HEC Approved PHD':pwr_HEC_approved,
                  'Highest Education': pwr_education,
                  'Email': pwr_email,
                  'Department' :pwr_department,
                  'extension' :pwr_extension,
                  'Image URL' : pwr_image_URL})

In [ ]:
df_pwr.tail()

,ID,Name,Designation,HEC Approved PHD,Highest Education,Email,Department,extension,Image URL
36,54,Mr. Osama Sohrab,Lecturer,No,M.S (Applied Mathematics),osama.sohrab@nu.edu.pk,sh,Ext 144,http://pwr.nu.edu.pk/images/faculty/Osama_Sohr...
37,53,Mr. Ikram Ullah,Assistant Professor,No,#,ikram.ullah@nu.edu.pk,sh,Ext 145,http://pwr.nu.edu.pk/images/faculty/Ikram_Ulla...
38,52,Mr. Askar Ali,Lecturer,No,#,askar.ali@nu.edu.pk,sh,Ext 145,http://pwr.nu.edu.pk/images/faculty/Askar_Ali1...
39,51,Ms. Noreen Shah,Lecturer,No,Ph.D. Scholar (Coursework completed),noreen.shah@nu.edu.pk,sh,Ext 150,http://pwr.nu.edu.pk/images/faculty/WhatsApp I...
40,50,Ms. Abida Farid,Lecturer,No,•\tMPhil in English Language & Communication f...,abida.farid@nu.edu.pk,sh,Ext 140,http://pwr.nu.edu.pk/images/faculty/Abida_Fari...


In [ ]:
df_pwr.to_csv('pwr.csv')

#CFD Campus

In [ ]:
# pip install fake-useragent


In [ ]:
import requests
from fake_useragent import UserAgent

# Create a UserAgent object to generate user-agent strings
ua = UserAgent()

# Generate a random user-agent string
user_agent = ua.random

# Define the URL you want to request
url = 'https://cfd.nu.edu.pk/all-departments/'  # Replace with the actual URL you want to request

# Create headers with the generated user-agent
headers = {
    'User-Agent': user_agent
}


response = requests.get(url, headers=headers)
print(response)

<Response [200]>


In [ ]:
soup_cfd = BeautifulSoup(response.content, 'html.parser')
# print(soup_cfd.prettify())

In [ ]:
links = soup_cfd.find_all('a', string="Faculty")
link = [tag['href'] for tag in links]
# print(link)

In [ ]:
cfd_id_ = []
cfd_name = []
cfd_designation = []
cfd_HEC_approved = []
cfd_education = []
cfd_email = []
cfd_department = []
cfd_extension = []
cfd_image_URL = []

In [ ]:
for i in range(len(link)):
  cfd_dept_req = requests.get(link[i] , headers=headers)
  soup_cfd_dept = BeautifulSoup(cfd_dept_req.content, 'html.parser')

  cfd_faculty = soup_cfd_dept.findAll('div' , class_='unitech-teacher')
  for j in range(len(cfd_faculty)):
    cfd_faculty_f = cfd_faculty[j]
    cfd_name.append(cfd_faculty_f.find('h4').text)          #name
    try:
      cfd_HEC_approved.append((cfd_faculty_f.find('p' , class_="hec").text).strip())
    except:
      cfd_HEC_approved.append("No")

    cfd_email.append(cfd_faculty_f.find('p').text)            #Email
    cfd_designation.append(cfd_faculty_f.find('h6').text)           #Designation

    cfd_dept_faculty_profile_req = requests.get(cfd_faculty_f.find('a')['href'] , headers=headers) #send request to server
    soup_cfd_dept_faculty_profile_f = BeautifulSoup(cfd_dept_faculty_profile_req.content, 'html.parser')
    cfd_department.append((soup_cfd_dept_faculty_profile_f.find('ul' , class_="teacher__address").find('li').text).strip())        #Department
    Ext = (((soup_cfd_dept_faculty_profile_f.find('ul' , class_="teacher__address")).find_all('li')[-1].text).replace(" " , "")).replace("\n" , "")             #Extension try except
    if Ext=="":
      cfd_extension.append("None")
    else:
      cfd_extension.append(Ext)
    cfd_image_URL.append((soup_cfd_dept_faculty_profile_f.find('div' , class_="teacher__thumb").find('img')['src']))       #image
    cfd_id_.append(soup_cfd_dept_faculty_profile_f.find('link', attrs={'rel': 'shortlink'})['href'].split("=")[-1])        #ID
    try:
      education_info = (soup_cfd_dept_faculty_profile_f.findAll('div', class_="htc__skill__container progress__bar--2")[0].find('ul').find('li').text)
    except AttributeError:
      try:
        education_info = (soup_cfd_dept_faculty_profile_f.findAll('div', class_="htc__skill__container progress__bar--2")[0].find('p').text)
      except AttributeError:
        education_info = "#"

    cfd_education.append(education_info)



In [ ]:
df_cfd = pd.DataFrame({'ID':cfd_id_,
                  'Name':cfd_name,
                  'Designation':cfd_designation,
                  'HEC Approved PHD':cfd_HEC_approved,
                  'Highest Education': cfd_education,
                  'Email': cfd_email,
                  'Department' :cfd_department,
                  'extension' :cfd_extension,
                  'Image URL' : cfd_image_URL})

In [ ]:
df_cfd.head()

,ID,Name,Designation,HEC Approved PHD,Highest Education,Email,Department,extension,Image URL
0,6074,Dr. Muhammad Ahmad ...,Associate Professor & HOD,HEC approved PhD Supervisor,Doctor of Philosophy (Hyperspectral Imaging) (...,dr.ahmad@nu.edu.pk,Department of Computer Science,Ext:160,https://cfd.nu.edu.pk/wp-content/uploads/2020/...
1,4748,Dr. Shahzad Sarfraz ...,Professor,HEC approved PhD Supervisor,"Ph.D. in Remote Sensing Image Processing AIT, ...",shahzad.sarfraz@nu.edu.pk,Department of Computer Science,Ext:102,https://cfd.nu.edu.pk/wp-content/uploads/2019/...
2,507933,Dr. Anwar Shah ...,Assistant Professor,HEC approved PhD Supervisor,"PhD Computer Sciences, FAST-NUCES, Peshawar",Anwar.Shah@nu.edu.pk,Department of Computer Science,Ext:131,https://cfd.nu.edu.pk/wp-content/uploads/2022/...
3,4752,Dr. Bilal Khan ...,Assistant Professor,HEC approved PhD Supervisor,"Ph.D. (Computer Science & Engineering), Dongg...",khan.bilal@nu.edu.pk,Department of Computer Science,Ext:126,https://cfd.nu.edu.pk/wp-content/uploads/2019/...
4,4772,Dr. Hashim Yasin ...,Assistant Professor,HEC approved PhD Supervisor,"Ph.D. (Computer Science), University of Bonn, ...",hashim.yasin@nu.edu.pk,Department of Computer Science,Ext:165,https://cfd.nu.edu.pk/wp-content/uploads/2019/...


In [ ]:
df_cfd.to_csv('cfd.csv')


#Step 2

In [ ]:
dfs = [df , df_pwr , df_cfd]
df_final = pd.concat(dfs, ignore_index=True)
df_final.to_csv("fast_faculty.csv", index=False)

#Step 3

In [ ]:
faculty = pd.read_csv("fast_faculty.csv")

In [ ]:
sample_ = faculty.sample(frac=0.8)
sample_.to_csv("sample.csv")